In [1]:
# REST API 호출, 이미지 파일 처리에 필요한 라이브러리
import requests
import json
import io
import base64
from PIL import Image

# [내 애플리케이션] > [앱 키] 에서 확인한 REST API 키 값 입력
REST_API_KEY = ''  ## kakao karlo model 사용을 위한 키값, kakao developers의 자신의 app key를 입력하세요.

# 이미지 생성하기 요청
def t2i(text, batch_size):
    r = requests.post(
        'https://api.kakaobrain.com/v1/inference/karlo/t2i',
        json = {
            'prompt': {
                'text': text,
                'batch_size': batch_size
            }
        },
        headers = {
            'Authorization': f'KakaoAK {REST_API_KEY}',
            'Content-Type': 'application/json'
        }
    )
    # 응답 JSON 형식으로 변환
    response = json.loads(r.content)
    return response

# Base64 디코딩 및 변환
def stringToImage(base64_string, mode='RGBA'):
    imgdata = base64.b64decode(str(base64_string))
    img = Image.open(io.BytesIO(imgdata)).convert(mode)
    return img

In [3]:
import yaml
from flask import Flask, escape, request, send_file, jsonify
from werkzeug.utils import secure_filename
from PIL import Image
from cog import Path
from pathlib import Path as PPath
from diffusers import StableDiffusionPipeline
import torch,os,html
from google.cloud import translate_v2 as translate
def translate_text(target, text):
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    # dot or comma separated text
    translate_client = translate.Client()

    if isinstance(text, bytes):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)
    return html.unescape(result["translatedText"])
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './credentials.json'
model_id = "Linaqruf/anything-v3.0"
anyv3_pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
anyv3_pipe = anyv3_pipe.to("cuda")

app = Flask(__name__)

/tmp/ipykernel_1280871/3090237825.py:2: DeprecationWarning: 'flask.escape' is deprecated and will be removed in Flask 2.4. Import 'markupsafe.escape' instead.
  from flask import Flask, escape, request, send_file, jsonify


In [4]:
@app.route('/flask3/test', methods=['POST','GET'])
def test():
    formdata = request.form
    prompt = formdata.get('prompt')
    # 이미지 생성하기 REST API 호출
    response = t2i(prompt, 4)
    # 응답의 첫 번째 이미지 생성 결과 출력하기
    print(response.get("images")[0])
    result = stringToImage(response.get("images")[0].get("image"), mode='RGB')
    result.save('temp/result.png')
    return send_file(PPath('temp/result.png'), mimetype='image/png')

In [5]:
@app.route('/flask3/any', methods=['POST','GET'])
def anyv3():
    formdata = request.form
    prompt = formdata.get('prompt')
    print(prompt,"-----")
    image = anyv3_pipe(prompt).images[0]
    image.save("temp/anyv3_result.png")
    return send_file(PPath('temp/anyv3_result.png'), mimetype='image/png')

In [6]:
@app.route('/flask3/gct', methods=['POST','GET'])
def googleTranslate():
    formdata = request.form
    prompt = formdata.get('prompt').replace(".",",") # 점을 컴마로 변환
    result = []
    for tag in prompt.split(","): #컴마기준 스플릿
        result.append(translate_text('en', tag))
    result = ','.join(result) # 컴마로 합치기
    print(f'gct: {result}')
    return result

In [ ]:
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5002)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5002
 * Running on http://70.12.130.121:5002
Press CTRL+C to quit
13.124.65.156 - - [18/May/2023 23:29:38] "POST /flask3/gct HTTP/1.1" 200 -


gct: cute, high quality, dog
{'id': 'f43bc20a-146a-4929-bee2-1e3c66fdbca3_0', 'image': '...', 'nsfw': False, 'nsfw_score': 0.22924805}


13.124.65.156 - - [18/May/2023 23:29:45] "POST /flask3/test HTTP/1.1" 200 -
13.124.65.156 - - [18/May/2023 23:33:32] "POST /flask3/gct HTTP/1.1" 200 -


gct: good dog
{'id': '8180fc53-6121-462f-aee0-6407279c7d13_0', 'image': '...', 'nsfw': False, 'nsfw_score': 0.20300293}


...

13.124.65.156 - - [19/May/2023 01:15:32] "POST /flask3/test HTTP/1.1" 200 -


{'id': '9e2499b2-c64b-4d53-92ef-dbfa8ddfb3d3_0', 'image': '...', 'nsfw': False, 'nsfw_score': 0.23144531}


13.124.65.156 - - [19/May/2023 01:23:57] "POST /flask3/gct HTTP/1.1" 200 -


gct: hi
{'id': 'd2b655f5-c86f-488d-9c50-033d34b0d109_0', 'image': '...', 'nsfw': False, 'nsfw_score': 0.23828125}


13.124.65.156 - - [19/May/2023 01:24:05] "POST /flask3/test HTTP/1.1" 200 -
